Example of genetic art code porting to Python.

In [1]:


import numpy as np

from quaternion import Quaternions
from op_map import formula_to_op_tree, OpDescr



In [2]:
# define image shape
height = 300
width = 300
depth = 1
img_shape = (width, height, 1)

In [3]:
formula_str = "( A2 ( mod golden x+iy ) ( /( +( +( A1 j y+k ) y+k ) y+k ) x+iy ) )"

In [4]:
op_tree = formula_to_op_tree(formula_str)

op_tree

(OpDescr(name='A2', degree=2, call_str='qaut2(a, b)', call_name='qaut2', depends_on_coords=False),
 (OpDescr(name='mod', degree=2, call_str='qmod(a, b)', call_name='qmod', depends_on_coords=False),
  OpDescr(name='golden', degree=0, call_str='qc5()', call_name='qc5', depends_on_coords=False),
  OpDescr(name='x_iy', degree=0, call_str='qcxy(x, y, z)', call_name='qcxy', depends_on_coords=True)),
 (OpDescr(name='/', degree=2, call_str='qdiv(a, b)', call_name='qdiv', depends_on_coords=False),
  (OpDescr(name='+', degree=2, call_str='qplus(a, b)', call_name='qplus', depends_on_coords=False),
   (OpDescr(name='+', degree=2, call_str='qplus(a, b)', call_name='qplus', depends_on_coords=False),
    (OpDescr(name='A1', degree=2, call_str='qaut1(a, b)', call_name='qaut1', depends_on_coords=False),
     OpDescr(name='j', degree=0, call_str='qc3()', call_name='qc3', depends_on_coords=False),
     OpDescr(name='y_k', degree=0, call_str='qcy1(x, y, z)', call_name='qcy1', depends_on_coords=True)),
   

In [5]:
op_tree[0]

OpDescr(name='A2', degree=2, call_str='qaut2(a, b)', call_name='qaut2', depends_on_coords=False)

In [6]:
# build image tensors
x_vec = np.linspace(start=0, stop=1, num=width)
y_vec = np.linspace(start=0, stop=1, num=height)
z_vec = np.zeros(1)

x = np.zeros(img_shape)
y = np.zeros(img_shape)
z = np.zeros(img_shape)

for xi in range(width):
    for yi in range(height):
        for zi in range(depth):
            x[xi][yi][zi] = x_vec[xi]
            y[xi][yi][zi] = y_vec[yi]
            z[xi][yi][zi] = z_vec[zi]

In [7]:
def r_dispatch(nd) -> Quaternions:
    """Evaluate formula over a tree"""
    q = Quaternions(img_shape)
    if isinstance(nd, OpDescr):
        method = getattr(q, nd.call_name)
        if nd.depends_on_coords:
            method(x, y, z)
        else:
            method()
    else:
        assert isinstance(nd[0], OpDescr)
        method = getattr(q, nd[0].call_name)
        args = [r_dispatch(nd[i]) for i in range(1, len(nd))]
        assert len(args) == nd[0].degree
        method(*args)
    return q
    



In [8]:
res = r_dispatch(op_tree)

In [9]:
res.k

array([[[ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        ...,
        [ 0.00000000e+00],
        [ 0.00000000e+00],
        [ 0.00000000e+00]],

       [[-2.99000000e+02],
        [ 1.79747006e+02],
        [ 3.39304475e+01],
        ...,
        [-3.99144332e-02],
        [-3.91207849e-02],
        [-3.83551306e-02]],

       [[-1.49500000e+02],
        [ 8.55847684e+01],
        [ 8.24713944e+01],
        ...,
        [-7.97406983e-02],
        [-7.81580430e-02],
        [-7.66310562e-02]],

       ...,

       [[-1.00673401e+00],
        [ 4.40718602e-01],
        [ 4.45392568e-01],
        ...,
        [ 6.71722273e-01],
        [ 6.72110636e-01],
        [ 6.72469376e-01]],

       [[-1.00335570e+00],
        [ 4.46326488e-01],
        [ 4.50982883e-01],
        ...,
        [ 6.70908660e-01],
        [ 6.71292121e-01],
        [ 6.71646051e-01]],

       [[-1.00000000e+00],
        [ 4.51856487e-01],
        [ 4.56494965e-01],
        ...,
        